In [2]:
import mteb
from sentence_transformers import SentenceTransformer
import os
import subprocess
from datasets import load_dataset

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import sys
sys.path.append('/root/MagicEmbed')

In [3]:
def show_dataset():
    # tasks = mteb.get_tasks(task_types=["PairClassification"],languages=["eng"])
    # tasks = mteb.get_tasks(task_types=["STS"],languages=["eng"])
    # tasks = mteb.get_tasks(languages=["eng"])
    # tasks = mteb.get_benchmark("MTEB(eng, classic)")
    tasks = mteb.get_tasks(languages=["eng"],
                           tasks=[
                "ArguAna",
                "NFCorpus",
                "SciFact",
                "StackOverflowDupQuestions",
                "SciDocsRR",
                "BiorxivClusteringS2S",
                "MedrxivClusteringS2S",
                "TwentyNewsgroupsClustering",
                "SprintDuplicateQuestions",
                "Banking77Classification",
                "EmotionClassification",
                "MassiveIntentClassification",
                "STS17",
                "SICK-R",
                "STSBenchmark",
                "SummEval"
            ]
                          )
    dataset_list = []
    for task in tasks:
        if task.metadata.name not in dataset_list:
            dataset_list.append(task.metadata.name)
            desc = 'name: {}\t\thf_name: {}\t\ttype: {}\t\tcategory: {}'.format(
                task.metadata.name, task.metadata.dataset['path'],
                task.metadata.type, task.metadata.category,
            )
            print(desc)
    print(len(dataset_list))
    print(dataset_list)
# show_dataset()

In [ ]:
data_path = '/root/MagicEmbed/task_assess/data'

def download_dataset():
    # tasks = mteb.get_tasks(task_types=["PairClassification"],languages=["eng"])
    # tasks = mteb.get_tasks(task_types=["STS"],languages=["eng"])
    tasks = mteb.get_tasks(languages=["eng"],
                           tasks=[
                "ArguAna",
                "NFCorpus",
                "SciFact",
                "StackOverflowDupQuestions",
                "SciDocsRR",
                "BiorxivClusteringS2S",
                "MedrxivClusteringS2S",
                "TwentyNewsgroupsClustering",
                "SprintDuplicateQuestions",
                "Banking77Classification",
                "EmotionClassification",
                "MassiveIntentClassification",
                "STS17",
                "SICK-R",
                "STSBenchmark",
                "SummEval"
            ]
            )
    err_list = []
    for task in tasks:
        # task.load_data()
        # https://huggingface.co/datasets/
        task_name = task.metadata.dataset['path']
        print(task_name)
        cmd = ['huggingface-cli', 'download', '--repo-type', 'dataset', '--resume-download',
               '--local-dir-use-symlinks', 'False', task_name, '--local-dir', os.path.join(data_path, task_name),
               ]
        try:
            result = subprocess.run(cmd, check=True)
        except subprocess.CalledProcessError as e:
            err_list.append(task_name)
            print("{} is error".format(task_name))

    if err_list:
        print('download failed: \n', '\n'.join(err_list))
    else:
        print('download success.')
# download_dataset()

In [5]:
# Define the sentence-transformers model name
# model_name = "average_word_embeddings_komninos"
# or directly from huggingface:

# tasks = mteb.get_tasks(tasks=["Banking77Classification"])
# evaluation = mteb.MTEB(tasks=tasks)
# results = evaluation.run(model, output_folder=f"results/{model_name}")

In [6]:
# os.environ['HF_DATASETS_CACHE'] = data_path
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# model_path = '/root/autodl-fs/hf-checkpoints/sentence-transformers/sentence-t5-base'
# model_path = '/root/autodl-fs/hf-checkpoints/sentence-transformers/sentence-t5-base'
# model_path = '/root/autodl-fs/hf-checkpoints/BAAI/bge-base-en-v1.5'
# model_path = '/root/autodl-fs/hf-checkpoints/Alibaba-NLP/gte-base-en-v1.5'
# model_path = '/root/autodl-fs/hf-checkpoints/hkunlp/instructor-base'
# model_path = '/root/autodl-fs/hf-checkpoints/princeton-nlp/sup-simcse-bert-base-uncased'
# model_path = '/root/autodl-fs/hf-checkpoints/WhereIsAI/UAE-Large-V1'
model_path = '/root/autodl-fs/hf-checkpoints/intfloat/e5-base'
model_name = os.path.basename(model_path)
model = SentenceTransformer(model_path,trust_remote_code=True)

In [7]:
import pandas as pd
def get_sticky_token_list(model_name):
    sticky_tokens_of_all_models_df = pd.read_csv("../results/final_all_models_sticky_tokens.csv")
    sticky_token_column = sticky_tokens_of_all_models_df[sticky_tokens_of_all_models_df['model'] == model_name]
    sticky_token_list = sticky_token_column['raw_vocab'].to_list()
    return sticky_token_list

In [8]:
sticky_tokens = get_sticky_token_list(model_name)
sticky_tokens

['human',
 'whoever',
 'carpet',
 'heating',
 'generating',
 'absorbing',
 'glide',
 'harvesting',
 'ochreous',
 'vibrating',
 'craving']

In [9]:
# from magicembed.utils import load_vocab_verifications
# token_infos_with_metrics_and_part_verifications = load_vocab_verifications(model_name)
# import pandas as pd
# import pygwalker as pyg
# df_token_infos_with_metrics_and_part_verifications = pd.DataFrame(token_infos_with_metrics_and_part_verifications).T
# df_strong_rejected_sorted = df_token_infos_with_metrics_and_part_verifications[df_token_infos_with_metrics_and_part_verifications['magic'] == 'strong_verified'].sort_values(by='max_prob', ascending=False)
# df_strong_rejected_sorted.head(5)
# token_list=df_strong_rejected_sorted['decoded'][:20].tolist()
# token_list

In [10]:
# tasks = mteb.get_tasks(task_types=["PairClassification"],languages=["eng"],tasks=['SprintDuplicateQuestions','TwitterSemEval2015','TwitterURLCorpus'])
# tasks = mteb.get_tasks(languages=["eng"],tasks=['BIOSSES', 'SICK-R', 'STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STSBenchmark','SprintDuplicateQuestions','TwitterSemEval2015','TwitterURLCorpus'])
# tasks = mteb.get_tasks(languages=["eng"],
#                            tasks=[
#                 "ArguAna",
#                 "NFCorpus",
#                 "SciFact",
#                 "StackOverflowDupQuestions",
#                 "SciDocsRR",
#                 "BiorxivClusteringS2S",
#                 "MedrxivClusteringS2S",
#                 "TwentyNewsgroupsClustering",
#                 "SprintDuplicateQuestions",
#                 "Banking77Classification",
#                 "EmotionClassification",
#                 "MassiveIntentClassification",
#                 "STS17",
#                 "SICK-R",
#                 "STSBenchmark",
#             ]
#             )
tasks = mteb.get_tasks(languages=["eng"],
                           tasks=[
                "ArguAna",
                "NFCorpus",
                "SciFact",
                "StackOverflowDupQuestions",
                "SciDocsRR",
                "BiorxivClusteringS2S",
                "MedrxivClusteringS2S",
                "TwentyNewsgroupsClustering",
                "SprintDuplicateQuestions",
                "Banking77Classification",
                "EmotionClassification",
                "MassiveIntentClassification",
                "STS16",
                "SICK-R",
                "STSBenchmark",
                "STS13",
                "SummEval"
            ]
            )
# for task in tasks:
#     # print(task.metadata.eval_splits)
#     print(os.path.join(data_path, task.metadata.dataset['path']))
#     # print(task.metadata.dataset['revision'])
#     # print(task.metadata.dataset.get('trust_remote_code', False))
#     dataset = load_dataset(
#             path=os.path.join(data_path, task.metadata.dataset['path']),
#             revision=task.metadata.dataset['revision'],
#             trust_remote_code=task.metadata.dataset.get('trust_remote_code', None),
#     )
#     # print(task.metadata_dict["dataset"])
#     task.dataset = dataset
#     task.data_loaded = True

In [11]:
# evaluation[2].dataset[0]['sent2']

In [12]:
tasks

MTEBTasks(ArguAna(name='ArguAna', languages=['eng']), NFCorpus(name='NFCorpus', languages=['eng']), SciFact(name='SciFact', languages=['eng']), StackOverflowDupQuestions(name='StackOverflowDupQuestions', languages=['eng']), SciDocsReranking(name='SciDocsRR', languages=['eng']), BiorxivClusteringS2S(name='BiorxivClusteringS2S', languages=['eng']), MedrxivClusteringS2S(name='MedrxivClusteringS2S', languages=['eng']), TwentyNewsgroupsClustering(name='TwentyNewsgroupsClustering', languages=['eng']), SprintDuplicateQuestionsPC(name='SprintDuplicateQuestions', languages=['eng']), Banking77Classification(name='Banking77Classification', languages=['eng']), EmotionClassification(name='EmotionClassification', languages=['eng']), MassiveIntentClassification(name='MassiveIntentClassification', languages=['eng']), STS16STS(name='STS16', languages=['eng']), SickrSTS(name='SICK-R', languages=['eng']), STSBenchmarkSTS(name='STSBenchmark', languages=['eng']), STS13STS(name='STS13', languages=['eng']), 

In [13]:
# tasks[0].data_loaded

In [14]:
# for split in tasks[1].dataset:
#     tasks[1].dataset[split] = tasks[1].dataset[split].map(lambda example: {"sentence2": [sentence + " hi" for sentence in example["sentence2"]]})

In [15]:
# tasks[0].dataset['test']['sentence2']

In [16]:
evaluation = mteb.MTEB(tasks=tasks)
results = evaluation.run(model, output_folder=f"/root/MagicEmbed/task_assess/results/{model_name}", verbosity=1)
# print(results)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- Banking77Classification, s2s

- EmotionClassification, s2s

- MassiveIntentClassification, s2s, multilingual 1 / 51 Subsets

...

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Scoring: 100%|██████████| 100/100 [00:01<00:00, 99.60it/s]


In [17]:
#retrieval 0-2
tasks[2].corpus['test']

{'4983': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging. Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion co

In [18]:
#reranking 3-4
tasks[4].dataset['test']['negative'][1]

['Analysis and Design of Average Current Mode Control Using a Describing-Function-Based Equivalent Circuit Model',
 'A Literature Review on Kidney Disease Prediction using Data Mining Classification Technique',
 'Shape-aware Instance Segmentation',
 'Research on continuous auditing: A bibliometric analysis',
 'Which Hotel attributes Matter ? A review of previous and a framework for future research',
 'Chain Replication for Supporting High Throughput and Availability',
 'Real-Time 3D Reconstruction and 6-DoF Tracking with an Event Camera',
 'BCSAT : A Benchmark Corpus for Sentiment Analysis in Telugu Using Word-level Annotations',
 'Dispositional Factors in Internet Use: Personality Versus Cognitive Style',
 'Protein function in precision medicine: deep understanding with machine learning.',
 'A framework based on RSA and AES encryption algorithms for cloud computing services',
 'Beyond engagement analytics: which online mixed-data factors predict student learning outcomes?',
 'Deep Voi

In [19]:
# tasks[-1].dataset['test']['sentence2']